Voreinstellung

In [1]:
#r "BoSSSpad.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
Init();


In [2]:
#r "StokesHelical_Ak.dll"

In [3]:
using StokesHelical_Ak;

Voreinstellungen

## Projekt Name

In [4]:
string proj = "Helical_HagenPoiseulle";

In [5]:
BoSSSshell.WorkflowMgm.Init(proj);

In [6]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [7]:
var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

## Loading job

In [8]:
myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [9]:
myDb

{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\akbari\cluster\Helical_HagenPoiseulle }

In [10]:
BoSSSshell.WorkflowMgm.Sessions

In [11]:
var Last_Session = BoSSSshell.WorkflowMgm.Sessions.Last();

Error: System.InvalidOperationException: Sequence contains no elements
   at System.Linq.ThrowHelper.ThrowNoElementsException()
   at System.Linq.Enumerable.Last[TSource](IEnumerable`1 source)
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Loading Field from Last Time Step

In [ ]:
var Pressure = Last_Session.Timesteps.Last().Fields.Single(f => f.Identification == "Pressure");
var ur = Last_Session.Timesteps.Last().Fields.Single(f => f.Identification == "ur");
var uxi = Last_Session.Timesteps.Last().Fields.Single(f => f.Identification == "uxi");
var ueta = Last_Session.Timesteps.Last().Fields.Single(f => f.Identification == "ueta");

## Exact Solution


In [ ]:
double MaxAmp = Last_Session.Controls.maxAmpli;
Func<double[], double> ur_exact = (X) => 0;
Func<double[], double> pressure_exact = (X) => 0;
Func<double[], double> ueta_exact = (X) => (X[0] / (Math.Sqrt(a * a * X[0] * X[0] + b * b))) * a * MaxAmp * X[0];
Func<double[], double> uxi_exact = (X) => (X[0] / (Math.Sqrt(a * a * X[0] * X[0] + b * b))) * b * MaxAmp;

In [ ]:
// Deep Copy
SinglePhaseField exSol_ur = ur.CloneAs();
SinglePhaseField exSol_ueta = ueta.CloneAs();
SinglePhaseField exSol_uxi = uxi.CloneAs();
SinglePhaseField exSol_pressure = Pressure.CloneAs();

In [ ]:
exSol_ur.ProjectField(ur_exact);
exSol_ueta.ProjectField(ueta_exact);
exSol_uxi.ProjectField(uxi_exact);
exSol_pressure.ProjectField(pressure_exact);

## L2-Error

In [ ]:
double ur_L2 = ur.L2Error(exSol_ur);
double uxi_L2 = uxi.L2Error(exSol_uxi);
double ueta_L2 = ueta.L2Error(exSol_ueta);
double pressure_L2 = Pressure.L2Error(exSol_pressure);

## Threshold

In [ ]:
NUnit.Framework.Assert.LessOrEqual(pressure_L2, 1e-9, "Pressure error to large");
NUnit.Framework.Assert.LessOrEqual(PressureL2Error, 1e-9, "Pressure error to large");
NUnit.Framework.Assert.LessOrEqual(PressureL2Error, 1e-9, "Pressure error to large");
NUnit.Framework.Assert.LessOrEqual(PressureL2Error, 1e-9, "Pressure error to large");